# Mandatory Assignment

### Torger Bocianowski

## Task 1

### Sets

$S=$ suppliers

$P=$ plants

### Parameters

$Sup_s=$ the supply for a supplier $s$.

$y_p=$ the yield per unit at a plant $p$.

$O_p=$ constant opening cost of a plant $p$.

$u_{sp}=$ unit cost from supplier $s$ to plant $p$.

$d_{sp}=$ distance driven (km) from supplier $s$ to plant $p$ by truck.

$C_t=$ truck loading / unloading cost ($\$10,000$)

$CapT_t=$ truck $t$'s capacity

$CapP_p=$ capacity at plant $p$.

### Decision Variables

$b_p\in\{0,1\}=$ binary variable, whether a plant at location $p$ should be built or not.

$x_{sp}=$ tons of biomass transported from supplier $s$ to plant $p$.

$t_{sp}\in\mathbb{Z}=$ integer variable, the number of trucks used from supplier to plant.

### Formulating the Objective Function & Constraints

_minimize Cost_: $\sum_{p\in P} O_pb_p$

$\hspace{4.9em}+\sum_{s\in S}\sum_{p\in P}(u_{sp}x_{sp}d_{sp}+C_tt_{sp})$

<br></br>

$\hspace{0.25em}$ _subject to_: $\space\sum_{p\in P}x_{sp}\cdot b_p\leq Sup_s\hspace{5em}\forall s\in S\hspace{5.9em}$ (Supply)

$\hspace{5em}\sum_{s\in S}\sum_{p\in P}x_{sp}\cdot y_p\leq500,000,000\hspace{7.9em}$ (Demand)

$\hspace{5em} x_{sp}\leq t_{sp}\cdot CapT_{sp}\hspace{6.5em}\forall s\in S,\forall p\in P\hspace{2.2em}$ (Number of trucks)

$\hspace{5em} \sum_{s\in S}x_{sp}\leq CapP_p\hspace{6em}\forall p\in P\hspace{5.6em}$ (Plant capacity)

<br></br>

### Programming

In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

gp.setParam("OutputFlag", 0)

# Dataframes
suppliers_df = pd.read_csv("data/suppliers.csv")
plants_df = pd.read_csv("data/plants.csv")
roads_df = pd.read_csv("data/roads_s_p.csv")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-24


### Sets

In [2]:
suppliers = suppliers_df.set_index("supplier")
suppliers = suppliers.to_dict(orient="index")

plants = plants_df.set_index("plant")
plants = plants.to_dict(orient="index")

### Defining the Model

In [3]:
m = gp.Model("SupplyChain")

### Parameters

In [4]:
def sp_param(df: pd.DataFrame, param: str) -> dict:
    df = df[["supplier", "plant", param]]
    df = df.set_index(["supplier", "plant"])
    df = df.to_dict(orient="index")
    return {(s, p): df[s, p][param] for s in suppliers for p in plants}

dist_s_p = sp_param(roads_df, "dist_s_p")
unit_cost_s_p = sp_param(roads_df, "cost_per_unit_s_p")
truck_cost_s_p = sp_param(roads_df, "truck_cost_s_p")
truck_cap_s_p = sp_param(roads_df, "truck_cap_s_p")

### Decision Variables

In [5]:
build = m.addVars(plants, vtype=GRB.BINARY, name="build")
biomass = m.addVars(suppliers, plants, name="biomass")
num_trucks = m.addVars(suppliers, plants, vtype=GRB.INTEGER, name="trucks")

### Objective Function

In [6]:
m.setObjective(
    gp.quicksum(
        plants[p]["plant_cost"] * build[p]
        for p in plants
    )
    + gp.quicksum(
        unit_cost_s_p[s, p] * biomass[s, p] * dist_s_p[s, p]
                            + (truck_cost_s_p[s, p] * num_trucks[s, p])
        for s in suppliers
        for p in plants
    ), 
    GRB.MINIMIZE,
)

### Constraints

In [7]:
DEMAND = 500_000_000

m.addConstrs(
    (
        gp.quicksum(
            biomass[s, p] for p in plants
        ) <= suppliers[s]["supply"]
        for s in suppliers
    ),
    name="supply",
)

m.addConstr(
    (
        gp.quicksum(
            biomass[s, p] * plants[p]["yield_per_unit"]
                    for s in suppliers 
                    for p in plants
        ) >= DEMAND
    ),
    name="demand",
)

m.addConstrs(
    (
        gp.quicksum(
            biomass[s, p] for s in suppliers
        ) <= plants[p]["plant_cap"] * build[p]
        for p in plants
    ),
    name="plant_cap",
)

m.addConstrs(
    (
        biomass[s, p] <= num_trucks[s, p] * truck_cap_s_p[s, p]
        for s in suppliers
        for p in plants
    ),
    name="num_trucks",
)

gp.setParam("OutputFlag", 0)

### Solving the Model

In [8]:
m.optimize()

In [9]:
locations = [p for p in plants if build[p].x == 1]
total_biomass = sum(biomass[s, p].x for s in suppliers for p in plants)
total_num_trucks = sum(num_trucks[s, p].x for s in suppliers for p in plants)

print("RESULTS\n")
print(f"({len(locations)}) Plants: {locations[:6]}")
print(f"\t     {locations[6:]}\n")
print(f"Total biomass: {total_biomass:,.0f} Mg")
print(f"Trucks needed: {total_num_trucks}")
print(f"\nTotal cost: ${m.objVal:,.0f}")


RESULTS

(11) Plants: [541, 9063, 9076, 9102, 9107, 9155]
	     [9178, 9183, 9203, 10058, 10066]

Total biomass: 2,155,172 Mg
Trucks needed: 4372.0

Total cost: $8,501,427,753


### Interpreting the Results

As shown above, 
- there are **11 locations** at which we want to **build plants**, 
- we need **4374 trucks** to transport **~2.155Mg** biomass and 
- the **total cost** is **~8.5 billion** dollars

## Task 2

### Sets
$S=$ suppliers

$H=$ hubs

$P=$ plants

### Parameters

$Sup_s=$ the supply for a supplier $s$.

$y_p=$ the yield per unit at a plant $p$.

$O_h=$ constant opening cost of a hub $h$.

$O_p=$ constant opening cost of a plant $p$.

$u_{sh}=$ unit cost from supplier $s$ to hub $h$.

$u_{hp}=$ unit cost from hub $h$ to plant $p$.

$d_{sh}=$ distance driven (km) from supplier $s$ to hub $h$ by truck.

$d_{hp}=$ distance driven (km) from hub $h$ to plant $p$ by train.

$C_t=$ truck loading / unloading cost ($\$10,000$)

$C_r=$ train loading / unloading cost ($\$60,000$)

$CapT_t=$ truck $t$'s capacity

$CapR_r=$ train $r$'s capacity

$CapH_h=$ capacity at hub $h$.

$CapP_p=$ capacity at plant $p$.

### Decision Variables

$b_h\in\{0,1\}=$ binary variable, whether a hub at location $h$ should be built or not.

$b_p\in\{0,1\}=$ binary variable, whether a plant at location $p$ should be built or not.

$x_{sh}=$ tons of biomass transported from supplier $s$ to hub $h$.

$x_{hp}=$ tons of biomass transported from hub $h$ to plant $p$.

$t_{sh}\in\mathbb{Z}=$ integer variable, the number of trucks to be used from supplier to hub.

$r_{hp}\in\mathbb{Z}=$ integer variable, the number of trains to be used from hub to plant.

### Formulating the Objective Function & Constraints

_minimize Cost_: $\sum_{h\in H} O_h\cdot b_h$

$\hspace{4.9em}+\sum_{p\in P} O_p\cdot b_p$

$\hspace{4.9em}+\sum_{s\in S}\sum_{h\in H}(u_{sh}x_{sh}d_{sh})+C_tt_{sh}$

$\hspace{4.9em}+\sum_{h\in H}\sum_{p\in P}(u_{hp}x_{hp}d_{hp})+C_rr_{hp}$

<br></br>

$\hspace{0.25em}$ _subject to_: $\space\sum_{h\in H}x_{sh}\cdot b_h\leq Sup_s\hspace{5em}\forall s\in S\hspace{5.9em}$ (Supply)

$\hspace{5em}\sum_{h\in H}\sum_{p\in P}x_{hp}\cdot y_p\leq500,000,000\hspace{7.9em}$ (Demand)

$\hspace{5em} x_{sh}\leq t_{sh}\cdot CapT_{sh}\hspace{6.5em}\forall s\in S,\forall p\in P\hspace{2.3em}$ (Number of trucks)

$\hspace{5em} x_{hp}\leq r_{hp}\cdot CapR_{hp}\hspace{6.2em}\forall h\in H,\forall p\in P\hspace{1.9em}$ (Number of trains)

$\hspace{5em} \sum_{s\in S}x_{sh}\leq CapH_p\hspace{6em}\forall h\in H\hspace{5.4em}$ (Hub capacity)

$\hspace{5em} \sum_{h\in H}x_{hp}\leq CapP_p\hspace{5.9em}\forall p\in P\hspace{5.6em}$ (Plant capacity)

$\hspace{5em} \sum_{s\in S}x_{sh}=\sum_{p\in P}x_{hp}\hspace{5em}\forall h\in H\hspace{5.4em}$ (Flow balance)

<br></br>

### Programming

### Sets

We add a new set hubs. The sets suppliers and plants remain the same from the previous task.

In [10]:
# Dataframes
hubs_df = pd.read_csv("data/hubs.csv")
railroads_df = pd.read_csv("data/railroads_h_p.csv")
roads_df = pd.read_csv("data/roads_s_h.csv")

# Sets
hubs = hubs_df.set_index("hub")
hubs = hubs.to_dict(orient="index")

### Defining the Model

In [11]:
m = gp.Model("SupplyChain2")

### Parameters

In [12]:
# Helper functions
def sh_param(df: pd.DataFrame, param: str) -> dict:
    df = df[["supplier", "hub", param]]
    df = df.set_index(["supplier", "hub"])
    df = df.to_dict(orient="index")
    return {(s, h): df[s, h][param] for s in suppliers for h in hubs}

def hp_param(df: pd.DataFrame, param: str) -> dict:
    df = df[["hub", "plant", param]]
    df = df.set_index(["hub", "plant"])
    df = df.to_dict(orient="index")
    return {(h, p): df[h, p][param] for h in hubs for p in plants}

# params from roads_s_h.csv
dist_s_h = sh_param(roads_df, "dist_s_h")
unit_cost_s_h = sh_param(roads_df, "cost_per_unit_s_h")
truck_cost_s_h = sh_param(roads_df, "truck_cost_s_h")
truck_cap_s_h = sh_param(roads_df, "truck_cap_s_h")

# params from railroads_h_p.csv
dist_h_p = hp_param(railroads_df, "dist_h_p")
unit_cost_h_p = hp_param(railroads_df, "cost_per_unit_h_p")
train_cost_h_p = hp_param(railroads_df, "train_cost_h_p")
train_cap_h_p = hp_param(railroads_df, "train_cap_h_p")

### Decision Variables

In [13]:
build_hub = m.addVars(hubs, vtype=GRB.BINARY, name="build_hub")
build_plant = m.addVars(plants, vtype=GRB.BINARY, name="build_plant")

biomass_s_h = m.addVars(suppliers, hubs, name="biomass_s_h")
biomass_h_p = m.addVars(hubs, plants, name="biomass_h_p")

num_trucks = m.addVars(suppliers, hubs, vtype=GRB.INTEGER, name="trucks")
num_trains = m.addVars(hubs, plants, vtype=GRB.INTEGER, name="trains")

### Objective Function

In [14]:
m.setObjective(
    gp.quicksum(
        hubs[h]["hub_cost"] * build_hub[h]
        for h in hubs
    )
    + gp.quicksum(
        plants[p]["plant_cost"] * build_plant[p]
        for p in plants
    )
    + gp.quicksum(
        unit_cost_s_h[s, h] * biomass_s_h[s, h] * dist_s_h[s, h] 
                        + (truck_cost_s_h[s, h] * num_trucks[s, h])
        for s in suppliers
        for h in hubs
    )
    + gp.quicksum(
        unit_cost_h_p[h, p] * biomass_h_p[h, p] * dist_h_p[h, p] 
                          + (train_cost_h_p[h, p] * num_trains[h, p])
        for h in hubs
        for p in plants
    ),
    GRB.MINIMIZE,
)

### Constraints

In [15]:
DEMAND = 500_000_000

m.addConstrs(
    (
        gp.quicksum(
            biomass_s_h[s, h] * build_hub[h] for h in hubs
        ) <= suppliers[s]["supply"]
        for s in suppliers
    ),
    name="supply",
)

m.addConstr(
    (
        gp.quicksum(
            biomass_h_p[h, p] * plants[p]["yield_per_unit"]
                    for h in hubs 
                    for p in plants
        ) >= DEMAND
    ),
    name="demand",
)

m.addConstrs(
    (
        biomass_s_h[s, h] <= num_trucks[s, h] * truck_cap_s_h[s, h]
        for s in suppliers
        for h in hubs
    ),
    name="num_trucks",
)

m.addConstrs(
    (
        biomass_h_p[h, p] <= num_trains[h, p] * train_cap_h_p[h, p]
        for h in hubs
        for p in plants
    ),
    name="num_trains",
)

m.addConstrs(
    (
        gp.quicksum(
            biomass_s_h[s, h] for s in suppliers
        ) <= hubs[h]["hub_cap"] * build_hub[h]
        for h in hubs
    ),
    name="hub_capacity",
)


m.addConstrs(
    (
        gp.quicksum(
            biomass_h_p[h, p] for h in hubs
        ) <= plants[p]["plant_cap"] * build_plant[p]
        for p in plants
    ),
    name="plant_capacity",
)

m.addConstrs(
    (
        gp.quicksum(biomass_s_h[s, h] for s in suppliers)
        == gp.quicksum(biomass_h_p[h, p] for p in plants)
        for h in hubs
    ),
    name="balance_flow",
)

gp.setParam("OutputFlag", 0)

### Solving the Model

In [16]:
m.optimize()

In [17]:
plants_locations = [p for p in plants if build_plant[p].x == 1]
hubs_locations = [h for h in hubs if build_hub[h].x == 1]

total_biomass_s_h = sum(biomass_s_h[s, h].x for s in suppliers for h in hubs)
total_biomass_h_p = sum(biomass_h_p[h, p].x for h in hubs for p in plants)

total_num_trucks = sum(num_trucks[s, h].x for s in suppliers for h in hubs)
total_num_trains = sum(num_trains[h, p].x for h in hubs for p in plants)

print("RESULTS\n")
print(f" ({len(plants_locations)}) Plants: {plants_locations}")
print(f"({len(hubs_locations)})   Hubs: {hubs_locations[:10]}")
print(f"\t     {hubs_locations[10:20]}")
print(f"\t     {hubs_locations[20:]}\n")
print(f"Total biomass: {total_biomass_s_h:,.0f} Mg") # biomass_s_h = biomass_h_p

print(f"Trucks needed: {total_num_trucks:.0f}")
print(f"Trains needed: {total_num_trains:.0f}")
print(f"\nTotal cost: ${m.objVal:,.0f}")

RESULTS

 (8) Plants: [541, 9047, 9060, 9091, 9178, 9183, 9203, 10066]
(31)   Hubs: [17201, 17218, 17359, 17372, 17395, 17404, 17447, 17466, 17507, 17592]
	     [17620, 17679, 17717, 17784, 17792, 17822, 17829, 17896, 17931, 17934]
	     [17942, 17943, 18029, 18042, 18063, 18082, 18127, 18286, 18288, 18294, 18303]

Total biomass: 2,155,172 Mg
Trucks needed: 4372
Trains needed: 124

Total cost: $5,135,420,807


### Interpreting the Results

As shown above, 
- there are **8 locations** at which we want to **build plants**, 
- there are **31 locations** at which we want to **build hubs**, 
- we need **4372 trucks** and **124** trains to transport **~2.155Mg** biomass and,
- the **total cost** is **~5.135 billion** dollars

## Task 3

### Sets

$S=$ suppliers

$H=$ hubs

$P=$ plants

$T=$ third party suppliers

### Parameters

$Sup_s=$ the supply for a supplier $s$.

$y_p=$ the yield per unit at a plant $p$.

$O_h=$ constant opening cost of a hub $h$.

$O_p=$ constant opening cost of a plant $p$.

$u_{sh}=$ unit cost from supplier $s$ to hub $h$.

$u_{hp}=$ unit cost from hub $h$ to plant $p$.

$u_{th}=$ unit cost from $t$ to $h$ ($\$2,000$).

$d_{sh}=$ distance driven (km) from supplier $s$ to hub $h$ by truck.

$d_{hp}=$ distance driven (km) from hub $h$ to plant $p$ by train.

$C_t=$ truck loading / unloading cost ($\$10,000$).

$C_r=$ train loading / unloading cost ($\$60,000$).

$CapT_t=$ truck $t$'s capacity.

$CapR_r=$ train $r$'s capacity.

$CapH_h=$ capacity at hub $h$.

$CapP_p=$ capacity at plant $p$.

### Decision Variables

$b_h\in\{0,1\}=$ binary variable, whether a hub at location $h$ should be built or not.

$b_p\in\{0,1\}=$ binary variable, whether a plant at location $p$ should be built or not.

$x_{sh}=$ tons of biomass transported from supplier $s$ to hub $h$.

$x_{hp}=$ tons of biomass transported from hub $h$ to plant $p$.

$x_{th}=$ tons of biomass transported from third party location $t$ to hub $h$.

$t_{sh}\in\mathbb{Z}=$ integer variable, the number of trucks to be used from supplier to hub.

$r_{hp}\in\mathbb{Z}=$ integer variable, the number of trains to be used from hub to plant.

### Formulating the Objective Function & Constraints

_minimize Cost_: $\sum_p O_h\cdot b_h$

$\hspace{4.9em}+\sum_p O_p\cdot b_p$

$\hspace{4.9em}+\sum_{sh}(u_{sh}x_{sh}d_{sh})+C_tt_{sh}$

$\hspace{4.9em}+\sum_{hp}(u_{hp}x_{hp}d_{hp})+C_rr_{hp}$

$\hspace{4.9em}+\sum_{th}(u_{th}x_{th}d_{th})$

<br></br>

$\hspace{0.25em}$ _subject to_: $\space\sum_{h}x_{sh}\cdot b_h\leq Sup_s\hspace{5.8em}\forall s\in S\hspace{4.3em}$ (Supply)

$\hspace{5em} \sum_{hp}x_{hp}\cdot y_p\leq500,000,000\hspace{9.3em}$ (Production)

$\hspace{5em} x_{sh}\leq t_{sh}\cdot C_{psh}\hspace{14.6em}$ (Number of trucks)

$\hspace{5em} x_{hp}\leq r_{hp}\cdot C_{rhp}\hspace{14.5em}$ (Number of trains)

$\hspace{5em} \sum_{s}x_{sh}\leq Cap_h\hspace{7.2em}\forall h\in H\hspace{4em}$ (Hub capacity)

$\hspace{5em} \sum_{h}x_{hp}\leq Cap_p\hspace{7.2em}\forall p\in P\hspace{4.1em}$ (Plant capacity)

$\hspace{5em} \sum_{s}x_{sh}+\sum_{t}x_{th}=\sum_{p}x_{hp}\hspace{2.5em}\forall h\in H\hspace{4em}$ (Flow balance)

<br></br>

### Programming

### Sets

Adding a set (in the same format) for the third party that emulates infinite supply.

In [18]:
third_party_supplier = {
    "x": {
        "supply": float("inf"),
    }
}

### Defining the Model

In [19]:
m = gp.Model("SupplyChain3")

### Parameters

The parameters stay the same from task 2, but we need to add the unit cost of $\$2000/Mg$

In [20]:
unit_cost_third_party = 2000

### Decision Variables

We add a new decision variable for the biomass from the third party supplier.

In [21]:
build_plant = m.addVars(plants, vtype=GRB.BINARY, name="build_plant")
build_hub = m.addVars(hubs, vtype=GRB.BINARY, name="build_hub")

num_trucks = m.addVars(suppliers, hubs, vtype=GRB.INTEGER, name="trucks")
num_trains = m.addVars(hubs, plants, vtype=GRB.INTEGER, name="trains")

biomass_s_h = m.addVars(suppliers, hubs, name="biomass_s_h")
biomass_h_p = m.addVars(hubs, plants, name="biomass_h_p")

third_party_biomass = m.addVars(third_party_supplier, hubs, name="third_party_biomass")

### Objective Function

In [22]:
m.setObjective(
    gp.quicksum(
        hubs[h]["hub_cost"] * build_hub[h]
        for h in hubs
    )
    + gp.quicksum(
        plants[p]["plant_cost"] * build_plant[p]
        for p in plants
    )
    + gp.quicksum(
        unit_cost_s_h[s, h] * biomass_s_h[s, h] * dist_s_h[s, h] 
                        + (truck_cost_s_h[s, h] * num_trucks[s, h])
        for s in suppliers
        for h in hubs
    )
    + gp.quicksum(
        unit_cost_h_p[h, p] * biomass_h_p[h, p] * dist_h_p[h, p] 
                          + (train_cost_h_p[h, p] * num_trains[h, p])
        for h in hubs
        for p in plants
    )
    + gp.quicksum(
        unit_cost_third_party * third_party_biomass[t, h]
        for t in third_party_supplier
        for h in hubs
    ),
    GRB.MINIMIZE,
)

### Constraints

The demand is now $800,000,000$. We must also alter the constraint for the hub capacity and the flow balance constraint. 

In [23]:
DEMAND = 800_000_000

m.addConstrs(
    (
        gp.quicksum(
            biomass_s_h[s, h] * build_hub[h] for h in hubs
        ) <= suppliers[s]["supply"]
        for s in suppliers
    ),
    name="supply",
)

m.addConstr(
    gp.quicksum(
        biomass_h_p[h, p] * plants[p]["yield_per_unit"]
                for h in hubs 
                for p in plants
        ) >= DEMAND,
    name="demand",
)

m.addConstrs(
    (
        biomass_s_h[s, h] <= num_trucks[s, h] * truck_cap_s_h[s, h]
        for s in suppliers
        for h in hubs
    ),
    name="trucks",
)

m.addConstrs(
    (
        biomass_h_p[h, p] <= num_trains[h, p] * train_cap_h_p[h, p]
        for h in hubs
        for p in plants
    ),
    name="trains",
)

m.addConstrs(
    (
        gp.quicksum(
            biomass_s_h[s, h] for s in suppliers
        ) + third_party_biomass[t, h] <= hubs[h]["hub_cap"] * build_hub[h]
        for h in hubs
        for t in third_party_supplier
    ),
    name="hub_capacity",
)

m.addConstrs(
    (
        gp.quicksum(
            biomass_h_p[h, p] for h in hubs
        ) <= plants[p]["plant_cap"] * build_plant[p]
        for p in plants
    ),
    name="plant_capacity",
)

m.addConstrs(
    (
        gp.quicksum(biomass_s_h[s, h] for s in suppliers) 
        + gp.quicksum(third_party_biomass[t, h] for t in third_party_supplier)
        == gp.quicksum(biomass_h_p[h, p] for p in plants)
        for h in hubs
    ),
    name="balance_flow",
)

m.setParam("OutputFlag", 0)

### Solving the Model

In [24]:
m.optimize()

In [25]:
plants_locations = [p for p in plants if build_plant[p].x == 1]
hubs_locations = [h for h in hubs if build_hub[h].x == 1]

total_biomass_s_h = sum(biomass_s_h[s, h].x for s in suppliers for h in hubs)
total_biomass_h_p = sum(biomass_h_p[h, p].x for h in hubs for p in plants)
total_biomass_third_party = sum(third_party_biomass[t, h].x for t in third_party_supplier for h in hubs)

total_num_trucks = sum(num_trucks[s, h].x for s in suppliers for h in hubs)
total_num_trains = sum(num_trains[h, p].x for h in hubs for p in plants)

print("RESULTS\n")
print(f" ({len(plants_locations)}) Plants: {plants_locations}")
print(f"({len(hubs_locations)})   Hubs: {hubs_locations[:9]}")
print(f"\t     {hubs_locations[9:]}")

print(f"\nTotal biomass sh:    {total_biomass_s_h:,.0f} Mg")
print(f"Total biomass hp:    {total_biomass_h_p:,.0f} Mg")
print(f"Total biomass third: {total_biomass_third_party:,.0f} Mg")

print(f"\nTrucks needed: {total_num_trucks:.0f}")
print(f"Trains needed: {total_num_trains:.0f}")
print(f"\nTotal cost: ${m.objVal:,.0f}")


RESULTS

 (6) Plants: [9044, 9047, 9178, 9183, 9203, 10066]
(18)   Hubs: [17201, 17218, 17359, 17447, 17507, 17592, 17679, 17792, 17829]
	     [17896, 17934, 17943, 18029, 18042, 18082, 18286, 18288, 18294]

Total biomass sh:    1,401,295 Mg
Total biomass hp:    3,448,276 Mg
Total biomass third: 2,046,981 Mg

Trucks needed: 2833
Trains needed: 179

Total cost: $7,139,972,964


### Interpreting the Results

As shown above, 
- there are **6 locations** at which we want to **build plants**, 
- there are **18 locations** at which we want to **build hubs**, 
- we need **2833 trucks** and **179** trains to transport **~3.448Mg** biomass and,
- the **total cost** is **~7.14 billion** dollars